In [41]:
import csv
import csv
import pandas as pd 

# This is our code for turning the Excel sheets containing round data into our Parameter CSV files used in our Optimization
# For this program to run properley, you must explicitly state the number of prior rounds and number of players that began the
# tournament. In addition, the Excel sheets you wish to read must be within the same directory. 

r = 7         # Number of overall rounds
n = 94        # Number of overall players
fileName = "round1"     #List of Excel sheets containing round data.

def xls2csv(fileName):
    xls_data = pd.read_excel(fileName+'.xlsx')
    i=0
    while True:
        if type(xls_data.iloc[i,[1]][0])==int:
            break
        else:
            i+=1
    xls_data = xls_data[i-1:]
    i=1
    while True:
        if type(xls_data.iloc[i,[1]][0])!=int:
            break
        else:
            i+=1
    xls_data = xls_data[:i]
    xls_data.columns = xls_data.iloc[0]
    xls_data=xls_data[1:]
    xls_data.to_csv(fileName+'.csv',index = False)
    return None
for aRound in range(1,r+1):
    xls2csv("round"+str(aRound))


xls2csv(fileName)

gijr = []

for r in range(1,r+1):
    for i in range(1,n+1):
        for j in range(1,n+1):
            if i == j:
                continue
            else:
                gijr.append("G%s,%d,%s" % (i,j,r))

wgir = []
bgir = []

for r in range(1,r+1):
    for i in range(1,n+1):
        wgir.append("WG%s,%s" % (i,r))
        bgir.append("BG%s,%s" % (i,r))
        
byegir = []

for r in range(1,r+1):
    for i in range(1,n+1):
        byegir.append("BYEG%s,%s" % (i,r))

WDir = []

for r in range(1,r+1):
    for i in range(1,n+1):
        WDir.append("WD%s,%s" % (i,r))
        
si = []

for i in range(1,n+1):
    si.append("S%s" % i)
        
gijr_values ={}
wgir_values = {}
bgir_values = {}
byegir_values = {}
WDir_values = {}
si_values = {}

for obj in gijr:
    gijr_values[obj] = 0
    
for obj in wgir:
    wgir_values[obj] = 0

for obj in bgir:
    bgir_values[obj] = 0
    
for obj in byegir:
    byegir_values[obj] = 0
    
for obj in WDir:
    WDir_values[obj] = 0
    
for obj in si:
    si_values[obj] = 0
    

listh = []
for ron in range(1,r+1):

    with open('round%s.csv'% ron) as csv_file:
        csv_reader = list(csv.reader(csv_file, delimiter=','))
    
        for row in csv_reader[1:]:
            i = row[1]
            j = row[17]
            if j == "":
                continue
                
            string = "G%s,%s,%d" % (i,j,ron)
            
            for obj in gijr:
                        
                if string == obj:
                    gijr_values[string] = 1
                                              
           
        for row in csv_reader[1:]:
            i = row[1]
            string1 = "WG%s,%d" % (i,ron)
            j = row[17] 
            string2 = "BG%s,%d" % (j,ron)
            k = row[12]
            
            for obj in wgir:
                if k == "bye" or k == "not paired":
                    continue
                    
                if string1 == obj:
                    wgir_values[string1] += 1 
                
            for obj in bgir:
        
                if string2 == obj:
                    bgir_values[string2] += 1 
                    
        for row in csv_reader[1:]:
            i = row[1]
            j = row[12]
            string1 = "BYEG%s,%d" % (i,ron)
            
            if j == "bye":
                byegir_values[string1] = 1
                
        for row in csv_reader[1:]:
            i = row[1]
            j = row[12]
            string1 = "WD%s,%d" % (i,ron)
            
            
            if j == "not paired" and string1.split(",")[0][2:] not in listh:
                WDir_values[string1] = 1
                listh.append(string1.split(",")[0][2:])
                
                
                
        for row in csv_reader[1:]:
            i = row[1]
            j = row[17]
            k = row[9]
            m = row[12]
            
            string1 = "S%s" % i
            string2 = "S%s" % j
            
            if k == "+ - -":
                si_values[string1] += 0.5
                si_values[string2] += 0.5
                
            elif k == "- - +":
                si_values[string2] += 0.5
                si_values[string1] += 0.5
            
            elif m == "bye":
                si_values[string1] +=1
                
            elif m == "not paired":
                si_values[string1] += 0
            
            else:
                k = row[9].split("-")
                
                if k[0] == 'Â½ ':
                    si_values[string1] += 0.5
                    si_values[string2] += 0.5
                
                else:
                    si_values[string1] += float(k[0])
                    si_values[string2] += float(k[1])

hold = [] 

with open('gijr1.csv', mode='w', newline='') as gijr_csv:
    gijr_writer = csv.writer(gijr_csv, delimiter=',')
    
    gijr_writer.writerow(["i","j","r","pairing"]) 
    
    for key,val in gijr_values.items():
        temp = key.split(",")
        
        gijr_writer.writerow([temp[0][1:],temp[1],temp[2],val])
        
with open('wgir1.csv', mode='w', newline='') as wgir_csv:
    wgir_writer = csv.writer(wgir_csv, delimiter=',')
    
    wgir_writer.writerow(["i","r","white"])
    
    for key,val in wgir_values.items():
        temp = key.split(",")
        wgir_writer.writerow([temp[0][2:],temp[1],val])
        
with open('bgir1.csv', mode='w', newline='') as bgir_csv:
    bgir_writer = csv.writer(bgir_csv, delimiter=',')
    
    bgir_writer.writerow(["i","r","black"])
    
    for key,val in bgir_values.items():
        temp = key.split(",")
        bgir_writer.writerow([temp[0][2:],temp[1],val])

with open('byegir1.csv', mode='w', newline='') as byegir_csv:
    byegir_writer = csv.writer(byegir_csv, delimiter=',')
    
    byegir_writer.writerow(["i","r","bye"])
    
    for key,val in byegir_values.items():
        temp = key.split(",")
        byegir_writer.writerow([temp[0][4:],temp[1],val])
        
with open('wdir1.csv', mode='w', newline='') as WDir_csv:
    WDir_writer = csv.writer(WDir_csv, delimiter=',')
    
    WDir_writer.writerow(["i","r","withdrawal"])
    
    for key,val in WDir_values.items():
        temp = key.split(",")
        WDir_writer.writerow([temp[0][2:],temp[1],val])
        
with open('si1.csv', mode='w', newline='') as si_csv:
    si_writer = csv.writer(si_csv, delimiter=',')
    
    si_writer.writerow(["i","score"])
    
    for key,val in si_values.items():
        
        si_writer.writerow([key[1:],val])
        
players = pd.read_csv('si1.csv')
players = players.set_index('i')
players = players.sort_values('score', ascending=False)
percentile_75 = players['score'][round(players.shape[0]*.25)]

aList = list(players['score']>=percentile_75)
aList = [1 if aList[i] else 0 for i in range(len(aList))]


players['Tp'] = aList
result = players['Tp']

result.to_csv("TPi1.csv", header=True)

In [15]:
"WD11,4".split(",")[0][2:] in listh

True